In [40]:
from selenium import webdriver
from random import *
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium.webdriver.common.keys import Keys
import re
import time
import unicodedata
#----------인스타그램 접속------------------
driver = webdriver.Chrome('./chromedriver.exe')
#한국관광공사 한글
url = 'https://www.instagram.com/kto9suk9suk/'
driver.get(url)

In [34]:
#첫 게시물 클릭
def select_first(driver):
    first=driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(randint(1,5))
Num=0
#게시물 데이터 크롤링
def get_content(driver):
    a=0
    while a<100:
        try:
            click=driver.find_element_by_css_selector("button.dCJp8.afkep")
            click.click()
            time.sleep(randint(2,7))
        except:
            break
        a+=1
#----------------이 까지 댓글 수 받아오기----------------------
    html=driver.page_source
    soup=bs(html, 'lxml')
    comment=soup.select('.Mr508')
    data=len(comment)
    try:
        content=soup.select('div.C4VMK>span')[0].text
        hashtags=re.findall(r'#[^\s#,\\]+', content)
        content_edit=unicodedata.normalize('NFC',content)
    except:
        content=' '
        hashtags=' '
        content_edit= ' '
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
#좋아요 개수
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
        see = 0
    except:
        like = 0
        see = soup.select('.vcOH2')[0].text[3:-1]   
#리스트에 저장
    data=[content_edit,date,like,see,hashtags,len(comment)]
    return data
#다음 게시물로 이동
def move_next(driver):
    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(randint(2,5))


In [38]:
#크롤링 실행
select_first(driver)
results=[]
iteration=2099
for i in range(iteration):
    try:
        data=get_content(driver)
        results.append(data)
        move_next(driver)
#         print(results[i][1],results[i][2],results[i][5])
    except:
        time.sleep(randint(2,5))
        move_next(driver)
#         print(results[i][1],results[i][2],results[i][5])

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a.coreSpriteRightPaginationArrow"}
  (Session info: chrome=87.0.4280.88)


In [39]:
#저장
df=pd.DataFrame(results)
df.columns=['내용','게시일','좋아요 수','영상 조회 수','해시태그','댓글 수']
df
df.to_excel('Instagram_Crawling_KTO_Korean_2.xlsx',encoding='CP949')